**First Step**
Import Libraries


In [17]:
import requests
from datetime import datetime, timedelta
from config import API_KEY
from flask import Flask, render_template_string
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

# API URLs
market_status_url = f'https://api.polygon.io/v1/marketstatus/now?apiKey={API_KEY}'
market_holiday_url = f'https://api.polygon.io/v1/marketstatus/upcoming?apiKey={API_KEY}'

def market_open_closed():
    response = requests.get(market_status_url)
    data = response.json()

    market_closed = data['market'] == 'closed'
    market_close_reason = ""

    if market_closed:
        today = datetime.now()
        if today.weekday() in [5, 6]: 
            market_close_reason = "Weekend"
        else:
            holidays = market_holiday() 
            holiday_names = [holiday['name'] for holiday in holidays if holiday['status'] == 'closed']
            if holiday_names:
                market_close_reason = f"The market is closed due to holiday(s): {', '.join(holiday_names)}."
          

    filtered_data = {
        'afterHours': data.get('afterHours', 'N/A'),
        'market': data.get('market', 'N/A'),
        's_and_p': data['indicesGroups'].get('s_and_p', 'N/A'),
        'nasdaq': data['indicesGroups'].get('nasdaq', 'N/A'),
        'dow_jones': data['indicesGroups'].get('dow_jones', 'N/A'),
        'ftse_russell': data['indicesGroups'].get('ftse_russell', 'N/A'),
        'closure_reason': market_close_reason
    }
    
    return filtered_data

def market_holiday():
    response = requests.get(market_holiday_url)
    data = response.json()
    return data

def next_holiday():
    holidays = market_holiday()
    today = datetime.now()
    next_month_date = today + timedelta(days=30)  
    upcoming = []

    for holiday in holidays:
        holiday_date = datetime.strptime(holiday['date'], '%Y-%m-%d')  
        if holiday['status'] == 'closed' and today <= holiday_date <= next_month_date:
            upcoming.append(holiday)

    return upcoming

def get_market_status():
    market_status = market_open_closed()
    upcoming_holidays = next_holiday()
    return market_status, upcoming_holidays


@app.route("/")
def index():
    market_status = market_open_closed()
    upcoming_holidays = next_holiday()

    html_content = """
    <!doctype html>
    <html lang="en">
      <head>
        <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1">
        <title>Is the Market Open or Closed</title>
        <link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
        <style>
          .market-status { width: 50px; height: 50px; border-radius: 50%; color: white; display: flex; align-items: center; justify-content: center; font-weight: bold; }
          .open { background-color: green; }
          .closed { background-color: red; }
        </style>
      </head>
      <body>
        <div class="container text-center my-5">
          <h1>Is the Market Open or Closed?</h1>
          <div class="row mt-5">
            <div class="col-md-3 col-sm-6 mb-4"><h5>S&P 500</h5><div class="market-status {{ 'open' if market_status['s_and_p'] == 'open' else 'closed' }} mx-auto">{{ market_status['s_and_p'].capitalize() }}</div></div>
            <div class="col-md-3 col-sm-6 mb-4"><h5>NASDAQ</h5><div class="market-status {{ 'open' if market_status['nasdaq'] == 'open' else 'closed' }} mx-auto">{{ market_status['nasdaq'].capitalize() }}</div></div>
            <div class="col-md-3 col-sm-6 mb-4"><h5>Dow Jones</h5><div class="market-status {{ 'open' if market_status['dow_jones'] == 'open' else 'closed' }} mx-auto">{{ market_status['dow_jones'].capitalize() }}</div></div>
            <div class="col-md-3 col-sm-6 mb-4"><h5>FTSE Russell</h5><div class="market-status {{ 'open' if market_status['ftse_russell'] == 'open' else 'closed' }} mx-auto">{{ market_status['ftse_russell'].capitalize() }}</div></div>
          </div>
          {% if market_status['closure_reason'] %}<p class="mt-3 text-danger">{{ market_status['closure_reason'] }}</p>{% endif %}
          <div class="mt-5"><h5>Upcoming Market Holidays in the Next 30 Days:</h5>
          {% if upcoming_holidays %}<ul class="list-unstyled">{% for holiday in upcoming_holidays %}<li>{{ holiday['name'] }} on {{ holiday['date'] }} (Exchange: {{ holiday['exchange'] }})</li>{% endfor %}</ul>{% else %}<p>No upcoming holidays within the next month.</p>{% endif %}
        </div>
      </body>
    </html>
    """
    return render_template_string(html_content, market_status=market_status, upcoming_holidays=upcoming_holidays)


app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
Exception in thread Thread-3:
Traceback (most recent call last):
  File "C:\Users\fboni\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connection.py", line 199, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\fboni\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "C:\Users\fboni\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\fboni\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py", line 789, in urlopen
 